# Assignment for Week 7

## Purpose          
This assignment is intended to help you do the following:      
  - Groupby operations to aggregate, transform, and filter data      
  - Built-in and custom user functions to perform groupby operations


## About the assignment

The function 'get_data_table' takes 4 parameters, df, date_col, group_col, type_x.      
  - The 'df' represents the dataframe.      
  -The 'date_col' represents the date column.      
  -The group_col represents the column to groupby and        
  -The 'type_x' represents the column with a specific value/category == 'type_x'. 

In [ ]:
def get_data_table(df, date_col, group_col, type_x):
    df_gross = (df.loc[(df.type == type_x) &
                                      df.category.isin(['category_x', 'category_xy', 'category_xyz'])
                                     ]
                         .groupby([group_col, date_col]).sum()
                         .unstack(date_col)['amount'].fillna(0)
                         .resample('M', axis=1).sum()
                        )

    df_recovered = (df.loc[(df.type == type_x) &
                                  df.category.isin(['category_01', 'category_05', 'category_07'])
                                 ]
                    .groupby([group_col, date_col]).sum()
                    .unstack(date_col)['amount'].fillna(0)
                    .resample('M', axis=1).sum()
                    )

    return df_gross.add(df_recovered * -1, fill_value=0)
    

## Tasks   
  1. Read the function above and in a 1-3 paragraphs explain its purpose    

  2. Apply the function to your selected dataframe and modify as needed based on your preferences.    

  3. Write 1-3 paragraphs to explain the differences or challenges when you applied the function to your chosen dataframe.

  4. Post your version of the above code on your GitHub account. Do not forget a README file with explanations.




**Action Item 1 - Expalin the initial function above.**

In pandas "def" is used to create a user defined function. In this case "get_data_table" is our function. 

It has four parameters; df, date_col, group_col, and type_x.

The syntax "df_gross = (df.loc[(df.type == type x) & df.category.isin(['category_x', 'category_xy', 'category_xyz'])]" comes across with the following meaning to me:

This part of the syntx is taking a defined location "df.loc" (selection by index) and pulling out "type_x" from the category's  "df.category.isin" from category_x, category_xy, and category_xyz, then we are grouping (.groupby) that output by "group_col, "date_col, and then summing it ".sum()". The ".unstack" syntax unpacks those summed values from the groupby command and places them into a column called "amount" and the ".fillna(0)" syntax fills all the missing values with a "zero".

Next, the ".resample" syntx is a method for resampling of a "time series". The "M" deals with which side of the bin interval is closed, "M" denotes the right side. The axis value of 1 denotes output in "column" format, and the ".sum()" syntax indicates again that the column output is to be "summed" (totaled). 

The "df_recovered" dataframe is basically a repeat of the "df_gross" function except the index has changed and the column headings being used are now "category_01, category_05, and category_07". The ".groupby.sum()", the ".unstack(date_col)['amount'].fillna(0)", and the ".resample('M', axis=1).sum()" syntax all repeat their respective functions again from the initial function from above.

Finally, the syntax "return df_gross.add(df_recovered * -1, fill_value=0) takes the output of "df_gross" and adds the output of "df_recoverd" to it (combines them into one dataframe) the "fill_value=0" places a "zero" in for all missing values that may be present in the combined dataframe.




**Action Item 2 - Apply the above function to your selected datafame. Modifying the function to fit your selected dataframe.**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
import pandas.util.testing as tm
from pandas.util.testing import assert_frame_equal
drugdata = pd.read_csv('https://raw.githubusercontent.com/frankDATA612/data_612/master/State_Drug_Utilization_Data_2010/State_Drug_Utilization_Data_2010.csv')

drugdata.head(3)



,Unnamed: 0,Utilization Type,State,Labeler Code,Product Code,Package Size,Year,Quarter,Product Name,Units Reimbursed,Suppression Used,Number of Prescriptions,Total Amount Reimbursed,Medicaid Amount Reimbursed,Non Medicaid Amount Reimbursed,Quarter Begin,Quarter Begin Date,Latitude,Longitude,Location,NDC
0,128037,MCOU,MA,37205,137,5,2010,4,ARTIFICIAL,15.0,False,1,4.08,4.08,0.0,10/1,10/01/2010,31.1801,-91.8749,"(31.1801, -91.8749)",37205013705
1,491755,FFSU,FL,62756,763,88,2010,1,TORSEMIDE,255.0,False,8,103.33,103.33,0.0,1/1,01/01/2010,39.3498,-75.5148,"(39.3498, -75.5148)",62756076388
2,2568076,FFSU,UT,46987,323,11,2010,2,Kadian Ora,120.0,False,2,905.24,905.24,0.0,4/1,04/01/2010,44.2853,-99.4632,"(44.2853, -99.4632)",46987032311


In [ ]:
# Let's create a drop list initially so that we are working with a smaller dataframe
drop_list =['Unnamed: 0', 'Utilization Type', 'State', 'Labeler Code', 'Product Code', 'Package Size', 'Product Name', 'Units Reimbursed', 'Suppression Used',
            'Number of Prescriptions', 'Quarter Begin', 'Quarter Begin Date', 'Latitude', 'Longitude', 'Location', 'NDC']
drugdata.drop(drop_list, inplace=True, axis=1)
drugdata.head()


,Year,Quarter,Total Amount Reimbursed,Medicaid Amount Reimbursed,Non Medicaid Amount Reimbursed
0,2010,4,4.08,4.08,0.0
1,2010,1,103.33,103.33,0.0
2,2010,2,905.24,905.24,0.0
3,2010,4,312.22,306.22,6.0
4,2010,2,52.87,52.87,0.0


Now Let's rename the Medicaid Dollar Amount Columns to TotalAmount, MedAmount and NonMedAmount.

In [ ]:
# Rename the two Medicaid Columns to single word headings:
drugdata.rename(columns={'Total Amount Reimbursed':'TotalAmount','Medicaid Amount Reimbursed':'MedAmount','Non Medicaid Amount Reimbursed':'NonMedAmount'}, inplace=True)

drugdata.head()

,Year,Quarter,TotalAmount,MedAmount,NonMedAmount
0,2010,4,4.08,4.08,0.0
1,2010,1,103.33,103.33,0.0
2,2010,2,905.24,905.24,0.0
3,2010,4,312.22,306.22,6.0
4,2010,2,52.87,52.87,0.0


In [ ]:
# Let's try a groupby on Quarter showing the unstacking of TotalAmount with a Sum
import numpy as np

drugdata.groupby(['Year', 'Quarter']) [['TotalAmount']].sum()



TotalAmount
Year Quarter              
2010 1        6.394527e+08
     2        9.635157e+08
     3        8.726385e+08
     4        9.411984e+08

Next, let's try the function syntax on our 2010 State Drug Utilization Dataset.

In [ ]:
# Let's try the Week 7 Function on the 2010 State Drug Utilization Dataset

def get_MedDollar_data (drugdata, MedAmount, NonMedAmount, Quarter):
    MedAmountQtr4 = (drugdata.loc[(drugdata.Quarter == 4) &
                                          drugdata.category.isin(['MedAmount'])]
                .groupby([Quarter, MedAmount]).sum()
                .unstack(MedAmount)['Amount'].fillna(0)
                .resample('M', axis=1).sum())
    
                
                
    
    NonMedAmountQtr4 = (drugdata.loc[(drugdata.Quarter == 4) &
                                              drugdata.category.isin(['NonMedAmount'])]
                .groupby([Quarter, NonMedAmount]).sum()
                .unstack(NonMedAmount)['Amount'].fillna(0)
                .resample('M', axis=1).sum())
                
             
    
    return MedAmountQtr4.add(NonMedAmountQtr4 * -1, fill_value=0)

print(drugdata)
print()
print(get_MedDollar_data)

    

    
                                          
                               

        Year  Quarter  TotalAmount  MedAmount  NonMedAmount
0       2010        4         4.08       4.08          0.00
1       2010        1       103.33     103.33          0.00
2       2010        2       905.24     905.24          0.00
3       2010        4       312.22     306.22          6.00
4       2010        2        52.87      52.87          0.00
...      ...      ...          ...        ...           ...
156215  2010        2        41.15      41.15          0.00
156216  2010        4        16.48      16.48          0.00
156217  2010        4        32.24      30.24          2.00
156218  2010        4       152.68     146.68          6.00
156219  2010        1       896.33     705.82        190.51

[156220 rows x 5 columns]

<function get_MedDollar_data at 0x7fe10a69ff28>


The fact that I got a memory printout indicates that there is nothing preventing the syntax from running within Pandas; however, it doesn't indicate that it will run correctly against the "drugdata" dataframe. We see this below by the errors that were generated when I tried to pass the dataframe through the function (get-MedDollar_data).

In [ ]:
breakoutMed = drugdata.groupby('Year').TotalAmount.agg(get_MedDollar_data)
print(breakoutMed)

TypeError: ignored

The purpose of my function (get_MedDollar_data) was to pull the MedAmount dollar values for the 4th Quarter of 2010 and add it to the NonMedAmount dollar values for the 4th Quarter of 2010. This was to have generated an unstacked output of the combined dollar amounts of Medicaid dollars added to Non-Medicaid dollars showing a total sum for the 4th Quarter.

I failed in my attempt. I would require an example of how to write the syntax for a similiar problem in order to apply it to my dataset and get a favorable output.

In [ ]:
# Let's see a different view of this task
import numpy as np
import sys
import seaborn as sns

drugdata.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156220 entries, 0 to 156219
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Year          156220 non-null  int64  
 1   Quarter       156220 non-null  int64  
 2   TotalAmount   156220 non-null  float64
 3   MedAmount     156220 non-null  float64
 4   NonMedAmount  156220 non-null  float64
dtypes: float64(3), int64(2)
memory usage: 6.0 MB


In [ ]:
# Let's use a function to add two columns of data together
def add(MedAmount, NonMedAmount):
  return MedAmount + NonMedAmount
drugdata["MedAmount + NonMedAmount"] = add(drugdata["MedAmount"], drugdata["NonMedAmount"])
drugdata.head(5)


,Year,Quarter,TotalAmount,MedAmount,NonMedAmount,MedAmount + NonMedAmount
0,2010,4,4.08,4.08,0.0,4.08
1,2010,1,103.33,103.33,0.0,103.33
2,2010,2,905.24,905.24,0.0,905.24
3,2010,4,312.22,306.22,6.0,312.22
4,2010,2,52.87,52.87,0.0,52.87


So, here I added MedAmount and NonMedAmount together in the function I called "add", then I output the combined sum into a new column called "MedAmount + NonMedAmount".

I also left the column TotalAmount in the dataframe as a reference to the addition function "add" just to see if it actually worked.

In [ ]:
# Group by Year and Quarter and use ".describe" on the TotalAmount Column.
MedDollars_describe = drugdata.groupby(['Year','Quarter']).TotalAmount.describe()
print(MedDollars_describe)

                count          mean  ...        75%          max
Year Quarter                         ...                        
2010 1        34514.0  18527.343723  ...  1997.7725  26636673.33
     2        39491.0  24398.361005  ...  2154.6550  47016635.91
     3        40867.0  21353.133388  ...  2183.3250  40510291.45
     4        41348.0  22762.851187  ...  2236.6850  50339487.23

[4 rows x 8 columns]


Just some basic statistics on the TotalAmount Column using the ".describe' function with ".groupby" used to filter the output by Year, and Quarter.